In [9]:
import math, pickle, glob
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.pylab as pl
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['axes.linewidth'] = 0.8

saveformat = '.eps' # or ''
dpi = 1200
paramtitle = False

In [10]:
for workspace in ['exp1a_noisy', 'exp1a_oja']:

    label_dir = './data/{}/'.format(workspace)
    wild_dir  = './data/{}/*.pickle'.format(workspace)

    list_for_lpwise_dicts = []

    count = 0

    print(glob.glob(wild_dir))
    for data_path in glob.glob(wild_dir):
        with open(data_path, 'rb') as f:
            hyperdict  = pickle.load(f)
            lpwise_dict = pickle.load(f)
            list_for_lpwise_dicts.append((hyperdict, lpwise_dict))
            count += 1 

    print('{} data were loaded.'.format(count))

    algo = hyperdict['algo']
    T, T_save, loop, k, p = hyperdict['T'], hyperdict['T_save'] ,hyperdict['loop'], hyperdict['k'], hyperdict['p']
    sigma, delta = hyperdict['sigma'], hyperdict['delta']
    learning_param_list = np.asarray(hyperdict['lp_list'])

    #Gamma selection
    allgamma = [x[0]['gamma'] for x in list_for_lpwise_dicts]

    print([x[0]['gamma'] for x in list_for_lpwise_dicts])

    if algo == 'noisy':
        noisydict = list_for_lpwise_dicts
        noisy_param_list   = learning_param_list
    elif algo == 'oja':
        ojadict = list_for_lpwise_dicts
        oja_param_list   = learning_param_list
    else:
        raise NotImplementedError
    

print('Hyper Dictionary Sample: ')
for k,v in hyperdict.items():
    print('{}: {}'.format(k,v))

['./data/exp1a_noisy/101246.pickle', './data/exp1a_noisy/707861.pickle', './data/exp1a_noisy/279499.pickle', './data/exp1a_noisy/625474.pickle']
4 data were loaded.
[0.0, 5e-05, 3e-05, 1e-05]
['./data/exp1a_oja/374225.pickle', './data/exp1a_oja/980012.pickle', './data/exp1a_oja/828438.pickle', './data/exp1a_oja/561240.pickle']
4 data were loaded.
[3e-05, 0.0, 5e-05, 1e-05]
Hyper Dictionary Sample: 
T: 144000
T_save: 100
loop: 2
k: 5
p: 100
sigma: 0.15
gamma: 1e-05
delta: 1.0
algo: oja
lp_list: [5.00000000e-01 3.33333333e-01 1.25000000e-01 1.00000000e-01
 5.00000000e-02 3.33333333e-02 2.50000000e-02 1.66666667e-02
 1.25000000e-02 1.00000000e-02 5.00000000e-03 3.33333333e-03
 2.50000000e-03 1.66666667e-03 1.25000000e-03 1.00000000e-03
 8.33333333e-04 6.66666667e-04 5.55555556e-04 5.00000000e-04
 3.33333333e-04 2.50000000e-04 1.66666667e-04 1.25000000e-04
 1.04166667e-04]
rotate_type: True


## Convergence Graph

In [11]:
fig = plt.figure(figsize=(12,8),facecolor=(1, 1, 1))
markers = ['o','h','s','^']
sizes = [150,180,150,150]

TITLESIZE = 40
TICKSIZE  = 32
LABELSIZE = 36
LEGENDSIZE= 28

# Basic settings
#ub = 0.4

# For \delta plot
ub = 0.65

trim = False
sparselabel = False #0, 3, 7... 만 label 부여
save = True
paramtitle=False

lidx = 4

handletextpad = 0.1

<Figure size 864x576 with 0 Axes>

## Noisy Power Method

In [12]:
ax_noisy = fig.add_subplot(1, 2, 1)
colors = pl.cm.copper(np.linspace(0,0.8,len(noisydict)))
print(colors)

noisydict = sorted(noisydict, key=lambda v: (v[0]['p'],-v[0]['delta'],v[0]['gamma'],v[0]['k']))

for c, (hyperdict, datadict_list) in enumerate(noisydict):
    
    lp_xaxis = np.copy(noisy_param_list)

    # Extract Data
    dist_array = np.zeros((loop,len(lp_xaxis)))    
    for repeat,datadict in enumerate(datadict_list):

        for lp_idx,lp in enumerate(lp_xaxis):
            dist_array[repeat,lp_idx] = datadict[lp][-1] #np.average(datadict[lp][len(datadict[lp])//3:])
    
    #dist_array = np.flip(dist_array,axis=1) if algo=='oja' else dist_array
    
    # Set Label
    #label_str = r'$(\Gamma,\delta,p)=({:.0e},{:.1f},{})$'.format(hyperdict['gamma'],hyperdict['delta'],hyperdict['p'])
    label_str = r'$\Gamma={:0.0f}e-5$'.format(hyperdict['gamma']*100000)
    
    # For \delta plot
    # label_str = r'$\delta={:.1f}$'.format(hyperdict['delta'])
        
    avg = np.average(dist_array,axis=0)
    std = np.std(dist_array,axis=0) / math.sqrt(loop)
    
    print('label={}, min={}'.format(label_str,np.min(avg)))
    
    # Trim unconverged data
    
    if trim:
        if np.where(avg<ub)[0].size != 0:
            args = np.where(avg<ub)[0]
            #print(args)
            lp_xaxis = lp_xaxis[args]
            avg, std = avg[args], std[args]
        
    # Plot line
    if sparselabel:
        label_str = label_str if c%4==1 else '_nolegend_'
        
    lp_xaxis = lp_xaxis[lidx:]
    avg = avg[lidx:]
    std = std[lidx:]
    
    ax_noisy.plot(lp_xaxis,avg,ls='--',linewidth=2,color=colors[c],zorder=1)
    ax_noisy.errorbar(lp_xaxis,avg,std,fmt='none', ecolor=colors[c],elinewidth=1,capsize=7,zorder=2)
    
    # Plot marker 
    signlist = np.ones(len(lp_xaxis))
    arg = np.argmin(avg)
    signlist[arg+1:] = -1
    for jdx, sign in enumerate(signlist):
        marker_s = (markers[c],sizes[c]) if sign==1 else (markers[c],sizes[c]*0.75)
        color_s  = ('none',colors[c]) if sign==1 else (colors[c],'white')
        if jdx == 0:
            ax_noisy.scatter(lp_xaxis[jdx],avg[jdx],marker=marker_s[0],s=marker_s[1],color=color_s[1],edgecolors=color_s[0],linewidths=2,\
                           label=label_str,zorder=3)
        else:
            ax_noisy.scatter(lp_xaxis[jdx],avg[jdx],marker=marker_s[0],s=marker_s[1],color=color_s[1],edgecolors=color_s[0],linewidths=2,zorder=3)
            
if paramtitle:
    title = 'Noisy Power Method'+r', $(\delta,\,\sigma,\,p/k)=({:.1f},\,{:.2f},\,{:d})$'.format(delta, sigma, p//k)
    titlesize = 28
else:
    title = 'Noisy Power Method'
    titlesize = TITLESIZE
ax_noisy.set_title(title,fontsize=titlesize,pad=15)
        
ax_noisy.set_xlim(10,20000)
ax_noisy.set_xticks([10.0,100.0,1000.0,10000,10000,10000])
ax_noisy.set_xticklabels([r'$10^{1}$',r'$10^{2}$',r'$10^{3}$',r'$10^{4}$','',''],fontsize=TICKSIZE)
ax_noisy.set_ylim(0.00,ub)
ax_noisy.set_yticks([0.0,0.2,0.4,0.6])
ax_noisy.set_yticklabels(['0','0.2','0.4','0.6'],fontsize=TICKSIZE)

ax_noisy.set_xlabel(r'Block Size $B$',fontsize=LABELSIZE,labelpad=5)
#ax_noisy.set_ylabel(r'$||\,U_{k\!\!+\!\!1\!:\!p}\,\,{\hat{U}_{1\!:\!k}^{\,\top}}\,||_2$',fontsize=28,labelpad=15)
ax_noisy.set_ylabel(r'$d({U}_{1\!:\!k}\,,\hat{U}_{1\!:\!k})$',fontsize=LABELSIZE,labelpad=15)
ax_noisy.set_xscale("log")

ax_noisy.legend(loc="upper right", fontsize=LEGENDSIZE,ncol=1,handletextpad=handletextpad,frameon=True,edgecolor='black')#,bbox_to_anchor=(-0.04,0.26))

[[0.         0.         0.         1.        ]
 [0.32941169 0.20832    0.13266667 1.        ]
 [0.65882337 0.41664    0.26533333 1.        ]
 [0.98823506 0.62496    0.398      1.        ]]
label=$\Gamma=0e-5$, min=0.017501633841695532
label=$\Gamma=1e-5$, min=0.030897639495625892
label=$\Gamma=3e-5$, min=0.04342486152872328
label=$\Gamma=5e-5$, min=0.05542998228365992


## Oja's Algorithm

In [13]:
ax_oja   = fig.add_subplot(1, 2, 2)
colors = pl.cm.bone(np.linspace(0,0.85,len(ojadict)))
print(colors)

ojadict = sorted(ojadict, key=lambda v: (v[0]['p'],-v[0]['delta'],v[0]['gamma'],v[0]['k']))

for c, (hyperdict, datadict_list) in enumerate(ojadict):

    lp_xaxis = np.copy(oja_param_list)
    
    # Extract Data
    dist_array = np.zeros((loop,len(lp_xaxis)))    
    for repeat,datadict in enumerate(datadict_list):

        for lp_idx,lp in enumerate(lp_xaxis):
            dist_array[repeat,lp_idx] = datadict[lp][-1]
    
    #dist_array = np.flip(dist_array,axis=1) if algo=='oja' else dist_array
    
    # Set Label
    # label_str = r'$(\Gamma,\delta,p)=({:.0e},{:.1f},{})$'.format(hyperdict['gamma'],hyperdict['delta'],hyperdict['p'])
    label_str = r'$\Gamma={:0.0f}e-5$'.format(hyperdict['gamma']*100000)
    
    # For \delta plot
    # label_str = r'$\delta={:.1f}$'.format(hyperdict['delta'])
        
    avg = np.average(dist_array,axis=0)
    std = np.std(dist_array,axis=0) / math.sqrt(loop)
    
    print('label={}, min={}'.format(label_str,np.min(avg)))
    
    # Trim unconverged data
    
    if trim:
        if np.where(avg<ub)[0].size != 0:
            args = np.where(avg<ub)[0]
            #print(args)
            lp_xaxis = lp_xaxis[args]
            avg, std = avg[args], std[args]
        
    # Plot line
    if sparselabel:
        label_str = label_str if c%4==1 else '_nolegend_'
        
    lp_xaxis = lp_xaxis[lidx:]
    avg = avg[lidx:]
    std = std[lidx:]
    
    ax_oja.plot(lp_xaxis,avg,ls='--',linewidth=2,color=colors[c],zorder=1)
    ax_oja.errorbar(lp_xaxis,avg,std,fmt='none', ecolor=colors[c],elinewidth=1,capsize=7,zorder=2)
    
    signlist = np.ones(len(lp_xaxis))
    arg = np.argmin(avg)
    signlist[arg+1:] = -1
    for jdx, sign in enumerate(signlist):
        marker_s = (markers[c],sizes[c]) if sign==1 else (markers[c],sizes[c]*0.75)
        color_s   = ('none',colors[c]) if sign==1 else (colors[c],'white')
        if jdx == 0:
            ax_oja.scatter(lp_xaxis[jdx],avg[jdx],marker=marker_s[0],s=marker_s[1],color=color_s[1],edgecolors=color_s[0],linewidths=2,\
                           label=label_str,zorder=3)
        else:
            ax_oja.scatter(lp_xaxis[jdx],avg[jdx],marker=marker_s[0],s=marker_s[1],color=color_s[1],edgecolors=color_s[0],linewidths=2,zorder=3)


if paramtitle:
    title = 'Oja\'s Algorithm'+r', $(\delta,\,\sigma,\,p/k)=({:.1f},\,{:.2f},\,{:d})$'.format(delta, sigma, p//k)
    titlesize = 28
else:
    title = 'Oja\'s Algorithm'
    titlesize = TITLESIZE
ax_oja.set_title(title,fontsize=titlesize,pad=15)
        
ax_oja.set_xlim(1.0/20000.0,1.0/10.0)
ax_oja.set_xticks([1.0e-4,1.0e-3,1.0e-2,1.0e-1,1.0e-1,1.0e-1])
ax_oja.set_xticklabels([r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{1}$','',''],fontsize=TICKSIZE)
ax_oja.set_xscale("log")
ax_oja.invert_xaxis()
ax_oja.set_ylim(0.00,ub)
ax_oja.set_yticks([0.0,0.2,0.4,0.6])
ax_oja.set_yticklabels(['','','',''],fontsize=TICKSIZE)

ax_oja.set_xlabel(r'Learning Rate $\zeta$',fontsize=LABELSIZE,labelpad=5)

ax_oja.legend(loc="upper left", fontsize=LEGENDSIZE,ncol=1,handletextpad=handletextpad,frameon=True,edgecolor='black')

[[0.         0.         0.         1.        ]
 [0.24705882 0.24705873 0.34373402 1.        ]
 [0.49754902 0.56433819 0.62254893 1.        ]
 [0.79626218 0.86960784 0.86960781 1.        ]]
label=$\Gamma=0e-5$, min=0.012542155038163628
label=$\Gamma=1e-5$, min=0.030967291532284485
label=$\Gamma=3e-5$, min=0.043331510598975505
label=$\Gamma=5e-5$, min=0.05540840068832599


In [14]:
for axis in ['top','bottom','left','right']:
    ax_oja.spines[axis].set_linewidth(1.5)
    ax_noisy.spines[axis].set_linewidth(1.5)

# E.O.D

In [15]:
fig.tight_layout()
fig.savefig('./'+'noisy_oja'+saveformat,dpi=dpi)
fig.savefig('./'+'noisy_oja'+'.png',dpi=dpi)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
